In [23]:
translation_names = ['WRJTx', 'WRJTy', 'WRJTz']
rot_names = ['WRJRx', 'WRJRy', 'WRJRz']
joint_names = [
    "R_thumb_MCP_joint1",
    "R_thumb_MCP_joint2",
    "R_thumb_PIP_joint",
    "R_thumb_DIP_joint",
    "R_index_MCP_joint",
    "R_index_DIP_joint",
    "R_middle_MCP_joint",
    "R_middle_DIP_joint",
    "R_ring_MCP_joint",
    "R_ring_DIP_joint",
    "R_pinky_MCP_joint",
    "R_pinky_DIP_joint"
]

In [24]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import numpy as np
from collections import defaultdict
import scipy.spatial.transform as transform
import numpy as np
import os
import transforms3d


def euler_to_rotation_matrix(euler_angles):
    rotation = R.from_euler('XYZ', euler_angles, degrees=False)
    return rotation.as_matrix()


def quaternion_to_rotation_matrix(quaternion):
    rotation = R.from_quat(quaternion)
    return rotation.as_matrix()


def object_pose_to_matrix(position, quaternion):
    """
    Converts object pose (position and quaternion) to a 4x4 transformation matrix.
    
    Parameters:
    - position: (3,) numpy.ndarray, position [x, y, z]
    - quaternion: (4,) numpy.ndarray, [x, y, z, w] quaternion
    
    Returns:
    - transformation_matrix: (4, 4) numpy.ndarray, corresponding transformation matrix
    """
    quaternion = np.concatenate([quaternion[1:4], quaternion[0:1]]) # xyzw -> wxyz
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = position
    return transformation_matrix


def hand_pose_to_matrix(position, quaternion):
    """
    Converts object pose (position and quaternion) to a 4x4 transformation matrix.
    
    Parameters:
    - position: (3,) numpy.ndarray, position [x, y, z]
    - quaternion: (4,) numpy.ndarray, [x, y, z, w] quaternion
    
    Returns:
    - transformation_matrix: (4, 4) numpy.ndarray, corresponding transformation matrix
    """
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = position
    return transformation_matrix

def get_global_pose_for_training(global_grasp_poses, oc_hand_pose, obj_idx):
    ''' Hand pose: trans + euler '''
    oc_hand_trans = oc_hand_pose[:3]
    oc_hand_orient = oc_hand_pose[3:]

    ''' Obj pose '''
    obj_pos = global_grasp_poses[obj_idx]['target_pose_world'][0].p
    obj_quat = global_grasp_poses[obj_idx]['target_pose_world'][0].q
    object_pose = object_pose_to_matrix(obj_pos, obj_quat)
    W_T_O = object_pose

    # Build O_T_H from object-centric pose
    R_oh = R.from_euler('xyz', oc_hand_orient, degrees=False).as_matrix()
    O_T_H = np.eye(4)
    O_T_H[:3, :3] = R_oh
    O_T_H[:3, 3]  = np.asarray(oc_hand_trans)

    # Back to world: W_T_H = W_T_O @ O_T_H
    W_T_H = W_T_O @ O_T_H

    # Extract
    R_wh = W_T_H[:3, :3]
    t_wh = W_T_H[:3, 3]
    euler_wh = R.from_matrix(R_wh).as_euler('XYZ', degrees=False)
    quat_wh  = R.from_matrix(R_wh).as_quat()  # (x,y,z,w)

    # 6D rotation like your forward function
    hand_6drot_world = R_wh[:, :2].T.ravel().tolist()

    # Hand pose in world frame
    hand_pose_world = t_wh.tolist() + euler_wh.tolist()

    return hand_pose_world

In [25]:
# load global grasp pose
global_grasp_poses_dict =np.load('/home/ubuntu/Documents/DexYCB/grasp_poses.npy', allow_pickle=True).item()

# load opt results
result_path = "/home/ubuntu/Documents/DexGraspNet/data/experiments/exp_2/results_real_v0"
refine_dict = defaultdict(dict)
e_pen_list = []
for fname in os.listdir(result_path):
    if fname.endswith(".npy"):
        # Remove extension and split at first underscore
        name_no_ext = os.path.splitext(fname)[0]
        object_idx, object_code = name_no_ext.split("_", 1)  # split only at first "_"
        object_idx = int(object_idx)
                
        # object centric hand pose
        opt_data_dict = np.load(os.path.join(result_path, name_no_ext + '.npy'), allow_pickle=True)[0]
        hand_qpos = opt_data_dict['qpos']
        E_pen = opt_data_dict['E_pen']
        e_pen_list.append([E_pen, object_idx])
        hand_trans = np.array([hand_qpos[name] for name in translation_names])
        hand_rot = np.array([hand_qpos[name] for name in rot_names])
        oc_hand_pose = np.concatenate([hand_trans, hand_rot])
        # rot = hand_rot[:, :2].T.ravel().tolist()
        # oc_hand_pose = [hand_qpos[name] for name in translation_names] + rot + [hand_qpos[name] for name in joint_names]

        # load global obj pose
        global_pose = get_global_pose_for_training(global_grasp_poses_dict, oc_hand_pose, object_idx)
        finger_joints = [hand_qpos[name] for name in joint_names]

        origin_map_idx = [0, 5, 10, 11, 1, 6, 2, 7, 3, 8, 4, 9]
        inv_map_idx = [0] * len(origin_map_idx)
        for new_pos, old_pos in enumerate(origin_map_idx):
            inv_map_idx[old_pos] = new_pos
        mapped_finger_joints = [finger_joints[i] for i in inv_map_idx]

        # single pose dict for testing:
        refine_dict[object_idx] = global_grasp_poses_dict[object_idx]
        refine_dict[object_idx]['robot_pose'] = [np.array(global_pose + mapped_finger_joints)]

e_pen_list = sorted(e_pen_list, key=lambda x: x[0])
for e_pen in e_pen_list:
    print("object_idx:", e_pen[1], "E_pen:", e_pen[0])

object_idx: 25 E_pen: 0.0
object_idx: 24 E_pen: 0.02102472260594368
object_idx: 2 E_pen: 0.03550920635461807
object_idx: 19 E_pen: 0.09591756761074066


In [26]:
refine_dict

defaultdict(dict,
            {25: {'target_object_name': '035_power_drill',
              'target_object_idx': 15,
              'target_pose_camera': [array([-0.888596  , -0.23289125,  0.24085154,  0.31328806,  0.26054224,
                      -0.25049844,  0.78198594], dtype=float32)],
              'target_pose_world': [Pose([0.489298, 0.266902, 0.346966], [-0.481551, -0.260079, -0.543124, -0.636776])],
              'camera_pose': Pose([1.01468, 0.777723, 0.799924], [0.0533614, -0.230272, -0.91078, 0.338536]),
              'robot_names': [<RobotName.inspire: 6>],
              'robot_pose': [array([ 0.40081137,  0.37438832,  0.34134235,  1.75713544,  0.45729249,
                      -0.3316439 ,  1.41893589,  0.8269974 ,  0.91402972,  0.84527212,
                       0.85544091,  0.28981453,  0.87675709,  0.88572621,  0.77593547,
                       0.77505094,  0.38588923,  0.2158599 ])],
              'hand_type': <HandType.right: 1>},
             2: {'target_object_nam

In [27]:
# store dict in a specified path as npy file
import os
import json
output_path = '/home/ubuntu/Documents/DexYCB/grasp_poses_opt_real_v0.npy'
if not os.path.exists(os.path.dirname(output_path)):
    os.makedirs(os.path.dirname(output_path))
np.save(output_path, refine_dict, allow_pickle=True)